In [48]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# SVM libraries
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC, SVR
from sklearn.datasets import load_digits
from sklearn.metrics import classification_report

# Read data set
data = pd.read_excel('Raisin_Dataset.xlsx', sheet_name = 'Raisin_Dataset')
data

# Data frame structure
data.info()

# Convert columns to numeric
data['Class'] = data['Class'].str.replace('Kecimen', '0')
data['Class'] = data['Class'].str.replace('Besni', '1')

cols = data.columns
data[cols] = data[cols].apply(pd.to_numeric, errors='coerce')

# Data frame structure
data.info()

# Classes are distrubuted evenly
data['Class'].value_counts()

# Data for SVM
X = data
X = X.drop(['Class'], axis = 1)
Y = data.Class # class

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             900 non-null    int64  
 1   MajorAxisLength  900 non-null    float64
 2   MinorAxisLength  900 non-null    float64
 3   Eccentricity     900 non-null    float64
 4   ConvexArea       900 non-null    int64  
 5   Extent           900 non-null    float64
 6   Perimeter        900 non-null    float64
 7   Class            900 non-null    object 
dtypes: float64(5), int64(2), object(1)
memory usage: 56.4+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 900 entries, 0 to 899
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Area             900 non-null    int64  
 1   MajorAxisLength  900 non-null    float64
 2   MinorAxisLength  900 non-null    float64
 3   Eccentricity     900 non-null    f

*ustawić gamma na sztywno
*pętla po k na zewnątrz
*potem dzielenie na zbiory
*potem pętla po C
*zapisuję max dokładność i C dla jakiej wyszła
*uśredniam dokładności
*wybieram dokładność najbliższej średniej i C dla niej

In [49]:
C = [1, 10, 100, 1000, 10000, 100000]
C = np.array(C)
#gamma = [0.0005, 0.005, 0.01, 0.05, 0.2, 0.8, 1.5, 2.5, 5, 10, 20, 50, 100]
#gamma = np.array(gamma)

Accuracy_CV2 = np.zeros((len(C),2))
Accuracy_rbf2 = np.zeros((10,2))

# SVM - rbf kernel - k is outside loop
for k in range(0, 10):
    # Split data into test and train sets
    X_train_rbf2, X_test_rbf2, Y_train_rbf2, Y_test_rbf2 = train_test_split(X, Y, test_size = 0.10) # 10 times cs so test_size is 10% of data set
    
    # Standarization
    sc = StandardScaler()
    sc.fit(X_train_rbf2)
    X_train_rbf = sc.transform(X_train_rbf2)
    X_test_rbf = sc.transform(X_test_rbf2)
    X_train_rbf = pd.DataFrame(X_train_rbf2)
    X_test_rbf = pd.DataFrame(X_test_rbf2)

    for i in range(0, len(C)):
        # Model
        svclassifier = SVC(kernel = 'rbf', C = C[i], gamma = 0.05) # gamma constant
        svclassifier.fit(X_train_rbf2, Y_train_rbf2)
        y_pred_rbf = svclassifier.predict(X_test_rbf2)
        
        # Accuracy
        Accuracy_CV2[i,0] = sum(y_pred_rbf == Y_test_rbf2)/len(Y_test_rbf2)
        #print("Accuarcy: ", Accuracy_CV2[i,0])
        Accuracy_CV2[i,1] = C[i]
        #print("C: ", Accuracy_CV2[i,1])

    Accuracy_rbf2[k,0] = np.max(Accuracy_CV2[:,0]) # best accuracy
    id_max = np.argmax(Accuracy_CV2[:,0], axis=0)
    #print("--------------------- ID MAX TO ---------------------", id_max)
    Accuracy_rbf2[k,1] = Accuracy_CV2[id_max,1] # C for best acuracy

print("Accuracy table:\n", Accuracy_rbf2) # display accuracy table

Accuracy table:
 [[0.48888889 1.        ]
 [0.45555556 1.        ]
 [0.43333333 1.        ]
 [0.48888889 1.        ]
 [0.44444444 1.        ]
 [0.46666667 1.        ]
 [0.48888889 1.        ]
 [0.4        1.        ]
 [0.37777778 1.        ]
 [0.5        1.        ]]


In [50]:
# Average accuracy
Accuracy_mean = np.mean(Accuracy_rbf2[:,0])
print("Mean accuracy =", Accuracy_mean)

Mean accuracy = 0.45444444444444443


In [53]:
# Accuracy closest to average accuracy
Accuarcy_roznica = Accuracy_rbf2[:,0] - Accuracy_mean
Accuarcy_roznica = abs(Accuarcy_roznica)
id_min = np.argmin(Accuarcy_roznica, axis=0)
avg_acc_C = Accuracy_rbf2[id_min, 1]
avg_acc = Accuracy_rbf2[id_min, 0]
print("C for value closest to average accuracy:",avg_acc_C)


C for value closest to average accuracy: 1.0
